# Bitácora para ensamblaje con Velvet
Tomado de [Holland Compunting Center](https://hcc.unl.edu/docs/guides/running_applications/bioinformatics_tools/de_novo_assembly_tools/velvet/running_velvet_with_paired_end_data/)

Zerbino, D.R., Birney, E., 2008. Velvet: Algorithms for de novo short read assembly using de Bruijn graphs. Genome Res. 18, 821–829. [doi:10.1101/gr.074492.107](http://www.genome.org/cgi/doi/10.1101/gr.074492.107)


### Esta bitácora considera que ya se realizó el análisis de calidad y se tienen los archivos obtenidos con Trim Galore

In [4]:
import os
from Bio import SeqIO, Entrez

In [5]:
cd ~/Desktop/bioinformatica_anotacion2019/data/sec_masiva/

/Users/migueldelrio/Desktop/bioinformatica_anotacion2019/data/sec_masiva


In [3]:
ls

8_S356_L001_R1_001.fastq        8_S356_L001_R2_001_fastqc.html
8_S356_L001_R1_001_fastqc.html  8_S356_L001_R2_001_fastqc.zip
8_S356_L001_R1_001_fastqc.zip   fastqc/
8_S356_L001_R2_001.fastq        velvet/


In [6]:
os.makedirs('velvet',exist_ok=True)

In [7]:
cd fastqc/

/Users/migueldelrio/Desktop/bioinformatica_anotacion2019/data/sec_masiva/fastqc


Tomado del manual de [Biopython](http://biopython.org/DIST/docs/tutorial/Tutorial.html#htoc292)

In [8]:
def cuentasecuencias(arch1):
    count, total_len, maximo = 0, 0, 0,
    minimo = 100
    for rec in SeqIO.parse(arch1, "fastq"):
        count += 1
        total_len += len(rec.seq)
        if maximo< len(rec.seq):
            maximo = len(rec.seq)
        if minimo > len(rec.seq):
            minimo = len(rec.seq)
    return (count, total_len, maximo, minimo)

In [9]:
arc = "archivo"
contar = "count"
tl = "total_len"
minimo = "minimo"
maximo = "maximo"
n = 1
arc1 = ["../8_S356_L001_R1_001.fastq", "8_S356_L001_R1_001_val_1.fq", "../8_S356_L001_R2_001.fastq", "8_S356_L001_R2_001_val_2.fq"]
for i in arc1:
    arc = "archivo"+str(n)
    contar = "count"+str(n)
    tl = "total_len"+str(n)
    minimo = "minimo"+str(n)
    maximo = "maximo"+str(n)
    
    
count1, count2, count3, count4 = 0, 0, 0, 0
total_len1, total_len2, total_len3, total_len4, = 0, 0, 0, 0
minimo1, minimo2, minimo3, minimo4 = 100, 100, 100, 100
maximo1, maximo2, maximo3, maximo4 = 0, 0, 0, 0
# cuenta secuencias en el primer archivo
archivo1 = "../8_S356_L001_R1_001.fastq"
count1, total_len1, maximo1, minimo1 = cuentasecuencias(archivo1)
archivo2 = "8_S356_L001_R1_001_val_1.fq"
count2, total_len2, maximo2, minimo2 = cuentasecuencias(archivo2)
archivo3 = "../8_S356_L001_R2_001.fastq"
count3, total_len3, maximo3, minimo3 = cuentasecuencias(archivo3)
archivo4 = "8_S356_L001_R2_001_val_2.fq"
count4, total_len4, maximo4, minimo4 = cuentasecuencias(archivo4)

print ("archivo              \t#sec \tmin max \t archivo-trim\t\t\t #   min max")
print(archivo1[2:17], "\t", count1, "\t", minimo1, maximo1, "\t", archivo2[:-3], "   ", count2, " ", minimo2, maximo2)
print(archivo3[2:17], "\t", count3, "\t", minimo3, maximo3, "\t", archivo4[:-3], "   ", count4, " ", minimo4, maximo4)


archivo              	#sec 	min max 	 archivo-trim			 #   min max
/8_S356_L001_R1 	 6652 	 35 251 	 8_S356_L001_R1_001_val_1     6642   24 251
/8_S356_L001_R2 	 6652 	 35 251 	 8_S356_L001_R2_001_val_2     6642   21 251


In [10]:
cd ../velvet/

/Users/migueldelrio/Desktop/bioinformatica_anotacion2019/data/sec_masiva/velvet


# Archivo de configuración para el ensamblador Velvet
Cambie los archivos de entrada, en este caso a:  
`8_S356_L001_R1_001_val_1.fq  
8_S356_L001_R2_001_val_2.fq`

Leer el [manual](https://www.ebi.ac.uk/~zerbino/velvet/Manual.pdf).  
Asimismo es necesario tomar en cuenta el nombre del archivo de la configuración. En este caso:  
`velvet_8_S356.sh`

In [12]:
fout = open("velvet_8_S356.sh", "w")
linea="""#!/bin/sh
#SBATCH --job-name=Velvet_Velveth
#SBATCH --nodes=1
#SBATCH --ntasks-per-node=8
#SBATCH --time=168:00:00
#SBATCH --mem=10gb
#SBATCH --output=Velveth.%J.out
#SBATCH --error=Velveth.%J.err

#module load velvet/1.2
#module load biokit
export OMP_NUM_THREADS=1

velveth output_directory/ 43 -fastq -longPaired -separate ../fastqc/8_S356_L001_R1_001_val_1.fq ../fastqc/8_S356_L001_R2_001_val_2.fq

"""

fout.write(linea)

fout.close()

### Verificando contenido del archivo de configuración

In [9]:
!head -50 velvet_8_S356.sh

#!/bin/sh
#SBATCH --job-name=Velvet_Velveth
#SBATCH --nodes=1
#SBATCH --ntasks-per-node=8
#SBATCH --time=168:00:00
#SBATCH --mem=10gb
#SBATCH --output=Velveth.%J.out
#SBATCH --error=Velveth.%J.err

#module load velvet/1.2
#module load biokit
export OMP_NUM_THREADS=$SLURM_NTASKS_PER_NODE

velveth output_directory/ 43 -fastq -longPaired -separate ../fastqc/8_S356_L001_R1_001_val_1.fq ../fastqc/8_S356_L001_R2_001_val_2.fq



In [10]:
ls

output_directory/ velvet_8_S356.sh  velvetg.sh


## Se ejecuta el comando `velveth`
Se utiliza el comando `time` para tener una idea del tiempo que tarda en ejecutarse.

In [13]:
!time sh velvet_8_S356.sh

[0.000000] Reading FastQ file ../fastqc/8_S356_L001_R1_001_val_1.fq;
[0.000155] Reading FastQ file ../fastqc/8_S356_L001_R2_001_val_2.fq;
[0.063850] 13284 sequences found in total in the paired sequence files
[0.063861] Done
[0.731592] Reading read set file output_directory//Sequences;
[0.737280] 13284 sequences found
[0.757009] Done
[0.757026] 13284 sequences in total.
[0.757225] Writing into roadmap file output_directory//Roadmaps...
[0.785780] Inputting sequences...
[0.785929] Inputting sequence 0 / 13284
[1.541051]  === Sequences loaded in 0.755284 s
[1.541262] Done inputting sequences
[1.541267] Destroying splay table
[1.579015] Splay table destroyed

real	0m1.692s
user	0m1.834s
sys	0m0.569s


### Una vez que se ha corrido `velveth`, es necesario ejecutar `velvetg`

# Archivo para ejecutar velvetg.
# Se debe cambiar el nombre del archivo generado, en este caso a:
`open("velvetg.sh", "w")`

In [14]:
fout = open("velvetg.sh", "w")
linea="""#!/bin/sh
#SBATCH --job-name=Velvet_Velvetg
#SBATCH --nodes=1
#SBATCH --ntasks-per-node=8
#SBATCH --time=168:00:00
#SBATCH --mem=100gb
#SBATCH --output=Velvetg.%J.out
#SBATCH --error=Velvetg.%J.err

module load velvet/1.2
#export OMP_NUM_THREADS=$SLURM_NTASKS_PER_NODE
export OMP_NUM_THREADS=1

velvetg output_directory/ -min_contig_lgth 200
"""

fout.write(linea)

fout.close()

In [15]:
!head -20 velvetg.sh

#!/bin/sh
#SBATCH --job-name=Velvet_Velvetg
#SBATCH --nodes=1
#SBATCH --ntasks-per-node=8
#SBATCH --time=168:00:00
#SBATCH --mem=100gb
#SBATCH --output=Velvetg.%J.out
#SBATCH --error=Velvetg.%J.err

module load velvet/1.2
#export OMP_NUM_THREADS=$SLURM_NTASKS_PER_NODE
export OMP_NUM_THREADS=1

velvetg output_directory/ -min_contig_lgth 200


## Se ejecuta el comando `velvetg`
Se utiliza el comando `time` para tener una idea del tiempo que tarda en ejecutarse.

In [16]:
!time sh velvetg.sh

velvetg.sh: line 10: module: command not found
[0.000000] Reading roadmap file output_directory//Roadmaps
[0.014292] 13284 roadmaps read
[0.014452] Creating insertion markers
[0.014995] Ordering insertion markers
[0.016206] Counting preNodes
[0.016690] 16811 preNodes counted, creating them now
[0.247499] Adjusting marker info...
[0.249038] Connecting preNodes
[0.255563] Cleaning up memory
[0.255591] Done creating preGraph
[0.255595] Concatenation...
[0.265615] Renumbering preNodes
[0.265629] Initial preNode count 16811
[0.266012] Destroyed 8371 preNodes
[0.266020] Concatenation over!
[0.266024] Clipping short tips off preGraph
[0.268663] Concatenation...
[0.269893] Renumbering preNodes
[0.269901] Initial preNode count 8440
[0.270201] Destroyed 1428 preNodes
[0.270209] Concatenation over!
[0.270212] 1025 tips cut off
[0.270216] 7012 nodes left
[0.270419] Writing into pregraph file output_directory//PreGraph...
[0.538369] Reading read set file output_directory//Sequences;
[0.550501] 1328

In [17]:
ls output_directory/

Graph       Log         Roadmaps    contigs.fa
LastGraph   PreGraph    Sequences   stats.txt


# Se verifican los archivos de salida y su contenido

## la estadística está en el archivo `stats.txt`

In [18]:
!wc -l output_directory/stats.txt

    5053 output_directory/stats.txt


In [19]:
!head -20 output_directory/stats.txt

ID	lgth	out	in	long_cov	short1_cov	short1_Ocov	short2_cov	short2_Ocov	short3_cov	short3_Ocov	short4_cov	short4_Ocov	long_nb	short1_nb	short2_nb	short3_nb	short4_nb
1	187	0	0	1.438503	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	2	0	0	0	0
2	181	0	0	1.988950	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	2	0	0	0	0
3	308	0	0	1.136364	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	2	0	0	0	0
4	113	2	0	1.079646	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	2	0	0	0	0
5	43	1	1	1.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	1	0	0	0	0
6	113	0	2	1.336283	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	2	0	0	0	0
7	43	1	1	1.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	1	0	0	0	0
8	279	0	0	1.487455	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	2	0	0	0	0
9	165	0	0	1.981818	0.000000	0.

In [20]:
!tail -20 output_directory/stats.txt

5033	100	0	0	1.980000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	2	0	0	0	0
5034	230	0	0	1.800000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	2	0	0	0	0
5035	344	0	0	1.136628	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	2	0	0	0	0
5036	287	0	0	1.090592	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	2	0	0	0	0
5037	190	0	0	1.326316	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	2	0	0	0	0
5038	43	1	1	1.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	1	0	0	0	0
5039	9	1	1	1.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	1	0	0	0	0
5040	34	1	2	2.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	2	0	0	0	0
5041	9	1	1	1.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	0.000000	1	0	0	0	0
5042	273	0	0	1.212454	0.000000	0.000000	0.000000	0.000000	0.000000	0.0

## mas información en el [manual](https://www.ebi.ac.uk/~zerbino/velvet/Manual.pdf).  



### Escriba en la siguiente celda qué contiene cada archivo.

### Verificando el contenido de contigs.fa

In [21]:
!head output_directory/contigs.fa

>NODE_1_length_187_cov_1.438503
GAATGAAATGTTCCAAAATGTCTTGGCATGCTGAAGCAGAGTTCCTTTCATCAGAACTAA
CGGGCCCAAATCCTGAAAAACAACCCCACACCATAACCCCCCCACCACCACCACCAAACT
TTAAACTTGATACAGTGCAATCAGACAAATACTGTTCTCCTGGCAACTGCCAAACCCAGA
CTCATCCAGACAGAGAAGTGTGACTGGTCACTCCAGAGAATATGTGTat
>NODE_2_length_181_cov_1.988950
GACTAAAGGCTGTCAACAAATTAATCAGTACAGCCTGTACCAAAAACAATTTTAGAATCT
GAAATTTGGCTCCCTGATAGCCTTGGATTGCTGGTTATCTCAGTTTTCTCCTGGATGTTT
TGTAAGCAGATGCTCTACCCCCCCCGCTGTCCTGTCTTCTGACCTGTGTGAACTGATATC
CCTAGACCAAGCTGCTGATGAACATTTCACATCTTATCAAAgt


# Contando el número de contigs

In [ ]:
!grep ">" output_directory/contigs.fa | wc -l

In [24]:
ls output_directory/

Graph       Log         Roadmaps    contigs.fa
LastGraph   PreGraph    Sequences   stats.txt


# Contando el número de secuencias en montaje (scaffolding)

In [25]:
!head output_directory/Graph

5052	13284	43	1
NODE	1	187	0	0	0	0	0	0	0	0
TCCTTTCATCAGAACTAACGGGCCCAAATCCTGAAAAACAACCCCACACCATAACCCCCCCACCACCACCACCAAACTTTAAACTTGATACAGTGCAATCAGACAAATACTGTTCTCCTGGCAACTGCCAAACCCAGACTCATCCAGACAGAGAAGTGTGACTGGTCACTCCAGAGAATATGTGTAT
GGATGAGTCTGGGTTTGGCAGTTGCCAGGAGAACAGTATTTGTCTGATTGCACTGTATCAAGTTTAAAGTTTGGTGGTGGTGGTGGGGGGGTTATGGTGTGGGGTTGTTTTTCAGGATTTGGGCCCGTTAGTTCTGATGAAAGGAACTCTGCTTCAGCATGCCAAGACATTTTGGAACATTTCATTC
NODE	2	181	0	0	0	0	0	0	0	0
AAAACAATTTTAGAATCTGAAATTTGGCTCCCTGATAGCCTTGGATTGCTGGTTATCTCAGTTTTCTCCTGGATGTTTTGTAAGCAGATGCTCTACCCCCCCCGCTGTCCTGTCTTCTGACCTGTGTGAACTGATATCCCTAGACCAAGCTGCTGATGAACATTTCACATCTTATCAAAGT
GGATATCAGTTCACACAGGTCAGAAGACAGGACAGCGGGGGGGGTAGAGCATCTGCTTACAAAACATCCAGGAGAAAACTGAGATAACCAGCAATCCAAGGCTATCAGGGAGCCAAATTTCAGATTCTAAAATTGTTTTTGGTACAGGCTGTACTGATTAATTTGTTGACAGCCTTTAGTC
NODE	3	308	0	0	0	0	0	0	0	0
AGTTCAGTGTGTTGCTCCTGTTCATCAGAGAGCAAGATATGTCTGTGCAAACACAGCTCCTCTCATGCAGGGACTCTTTGCTCCGCCCCCTCCTCTCCGCACCGCTGTGATTAGTTGTCTGCGGTCACGTGACTCAGCGGCACTGAGCAGTTATGGGCTGCAG

In [26]:
!head -30 output_directory/Log

Wed Feb 27 13:32:03 2019
 velveth output_directory/ 43 -fastq -longPaired -separate ../fastqc/8_S356_L001_R1_001_val_1.fq ../fastqc/8_S356_L001_R2_001_val_2.fq
Version 1.2.10
Copyright 2007, 2008 Daniel Zerbino (zerbino@ebi.ac.uk)
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.
Compilation settings:
CATEGORIES = 4
MAXKMERLENGTH = 191
OPENMP
LONGSEQUENCES

Wed Feb 27 13:37:40 2019
 velvetg output_directory/ -min_contig_lgth 200
Version 1.2.10
Copyright 2007, 2008 Daniel Zerbino (zerbino@ebi.ac.uk)
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.
Compilation settings:
CATEGORIES = 4
MAXKMERLENGTH = 191
OPENMP
LONGSEQUENCES

Final graph has 5068 nodes and n50 of 275, max 929, total 965477, using 6333/13284 reads
Wed Feb 27 14:57:09 2019
 velveth output_directory/ 43 -fastq -longPaired -s

In [27]:
!head output_directory/Roadmaps

13284	0	43	1
ROADMAP 1
ROADMAP 2
-1	1	185	103
ROADMAP 3
ROADMAP 4
-3	1	179	0
ROADMAP 5
ROADMAP 6
-5	99	208	166


In [57]:
!head output_directory/LastGraph

5068	13284	43	1
NODE	1	187	0	0	0	0	0	0	0	0
TCCTTTCATCAGAACTAACGGGCCCAAATCCTGAAAAACAACCCCACACCATAACCCCCCCACCACCACCACCAAACTTTAAACTTGATACAGTGCAATCAGACAAATACTGTTCTCCTGGCAACTGCCAAACCCAGACTCATCCAGACAGAGAAGTGTGACTGGTCACTCCAGAGAATATGTGTAT
GGATGAGTCTGGGTTTGGCAGTTGCCAGGAGAACAGTATTTGTCTGATTGCACTGTATCAAGTTTAAAGTTTGGTGGTGGTGGTGGGGGGGTTATGGTGTGGGGTTGTTTTTCAGGATTTGGGCCCGTTAGTTCTGATGAAAGGAACTCTGCTTCAGCATGCCAAGACATTTTGGAACATTTCATTC
NODE	2	181	0	0	0	0	0	0	0	0
AAAACAATTTTAGAATCTGAAATTTGGCTCCCTGATAGCCTTGGATTGCTGGTTATCTCAGTTTTCTCCTGGATGTTTTGTAAGCAGATGCTCTACCCCCCCCGCTGTCCTGTCTTCTGACCTGTGTGAACTGATATCCCTAGACCAAGCTGCTGATGAACATTTCACATCTTATCAAAGT
GGATATCAGTTCACACAGGTCAGAAGACAGGACAGCGGGGGGGGTAGAGCATCTGCTTACAAAACATCCAGGAGAAAACTGAGATAACCAGCAATCCAAGGCTATCAGGGAGCCAAATTTCAGATTCTAAAATTGTTTTTGGTACAGGCTGTACTGATTAATTTGTTGACAGCCTTTAGTC
NODE	3	308	0	0	0	0	0	0	0	0
AGTTCAGTGTGTTGCTCCTGTTCATCAGAGAGCAAGATATGTCTGTGCAAACACAGCTCCTCTCATGCAGGGACTCTTTGCTCCGCCCCCTCCTCTCCGCACCGCTGTGATTAGTTGTCTGCGGTCACGTGACTCAGCGGCACTGAGCAGTTATGGGCTGCAG

In [28]:
!head output_directory/PreGraph

7012	13284	43	1
NODE	1	187
GAATGAAATGTTCCAAAATGTCTTGGCATGCTGAAGCAGAGTTCCTTTCATCAGAACTAACGGGCCCAAATCCTGAAAAACAACCCCACACCATAACCCCCCCACCACCACCACCAAACTTTAAACTTGATACAGTGCAATCAGACAAATACTGTTCTCCTGGCAACTGCCAAACCCAGACTCATCCAGACAGAGAAGTGTGACTGGTCACTCCAGAGAATATGTGTAT
NODE	2	181
GACTAAAGGCTGTCAACAAATTAATCAGTACAGCCTGTACCAAAAACAATTTTAGAATCTGAAATTTGGCTCCCTGATAGCCTTGGATTGCTGGTTATCTCAGTTTTCTCCTGGATGTTTTGTAAGCAGATGCTCTACCCCCCCCGCTGTCCTGTCTTCTGACCTGTGTGAACTGATATCCCTAGACCAAGCTGCTGATGAACATTTCACATCTTATCAAAGT
NODE	3	308
ATATTAGCGTGATAGGATCGATACTTCAGTTTGTCTCCTGTAAGTTCAGTGTGTTGCTCCTGTTCATCAGAGAGCAAGATATGTCTGTGCAAACACAGCTCCTCTCATGCAGGGACTCTTTGCTCCGCCCCCTCCTCTCCGCACCGCTGTGATTAGTTGTCTGCGGTCACGTGACTCAGCGGCACTGAGCAGTTATGGGCTGCAGTGAGTGAGAAAGCCGAGCGTCGTGCAGCTGTACTTTACAGCTGGAAATGAAAACAGTGTTAACAGTGATGCGTTTTGGGAGGAAATTTTCTACTGTGGCAACACCATGGACTTATTTAAAGACATGAAAGTGCAGGAAAAAGAAT
NODE	4	113
TCTGTCTGCAGCAGCTCCAGAACCCTTCCAAGACCGGGAGGATGATCAGATCACATCTGTCATGTTCAAGTTCCGCTCGCCAAGTTCCTCTCTCTGCTTCGGTTCCTACATTGTAACCATGCCAGTGATGTCATT

In [29]:
!head output_directory/Sequences

>M00313:49:000000000-A41BL:1:1101:15868:1583	1	9
GNATGAAATGTTCCAAAATGTCTTGGCATGCTGAAGCAGAGTTCCTTTCATCAGAACTAA
CGGGCCCAAATCCTGAAAAACAACCCCACACCATAACCCCCCCACCACCACCACCAAACT
TTAAACTTGATACAGTGCAATCAGACAAATACTGTTCTCCTGGCAACTGCCAAACCCAGA
CTCATCCAGACAGAGAAGTGTGACTGGTCACTCCAGAGAATATGTGTA
>M00313:49:000000000-A41BL:1:1101:15868:1583	2	9
NTACACATATTCTCTGGAGTGACCAGTCACACTTCTCTGTCTGGATGAGTCTGGGTTTGG
CAGTTGCCAGGAGAACAGTATTTGTCTGATTGCACTGTATCAAGTTTAAAGTTTGGTGGT
GGTGGGGGGGGGGTTATGGTGTGGGGTTGTTTTTCAGGATTTGGGCCGGTTAGTTCTGAT
GAAAGGAACTCTGCTTCAGCATGCCAAGGCATTTGGGAAGATTTCATGCCTGGCTCTTAT


## Una vez concluido el ensamblaje se procede a la anotación de los contigs